In [1]:
from tqdm import tqdm
import tensorflow as tf
import json
import tensorflow_text as text
import os
import numpy as np
import pandas as pd
import pickle

In [2]:
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
#m_true=Chem.MolFromSmiles(df_final["decoded_test"][1])
#m_predict=Chem.MolFromSmiles(df_final["predicted"][1])
#ms=[m_true,m_predict]
#fps = [Chem.RDKFingerprint(x) for x in ms]
#DataStructs.FingerprintSimilarity(fps[0],fps[1])

def get_mf(smiles1):
    try:
        m_true=Chem.MolFromSmiles(smiles1)
        mf=CalcMolFormula(m_true)
        return mf
    except:
        return "no_prediction_or_error_with_parsing_by_rdkit"
    

In [3]:
root='C:/Users/delser/'
#name='all-HRMS-loss-header-train-refine_prepro.tsv'
#name='all_HRMS_train_24012023_cddd_refine_s.tsv'
#name='all_HRMS_valid_24012023_cddd_refine_s.tsv'
name='casmi_func_groups_2201.tsv'

os.path.join(root, name)
df_valid = pd.read_csv(os.path.join(root, name), sep="\t") 

df_valid

,Unnamed: 0,spectrum_id,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,smiles_preprocessed,num_of_sugars,Number of aliphatic carboxylic acids,...,Number of phenols,Number of phosphoric acid groups,Number of phosphoric ester groups,Number of piperdine rings,Number of primary amides,Number of pyridine rings,Number of quaternary nitrogens,Number of thioether,Number of thiazole rings,Number of unbranched alkanes of at least 4 members (excludes halogenated alkanes)
0,0,3,719.2538,"[53.0387, 55.0179, 55.0545, 57.0338, 59.0492, ...","[0.017272727272727273, 0.01818181818181818, 0....",[314.15709999999996],[0.01818181818181818],CC1C(C(C(C(O1)OC2=C(OC3=C(C(=CC(=C3C2=O)O)O)CC...,2.0,0,...,2,0,0,0,0,0,0,0,0,0
1,1,5,499.2298,"[67.0543, 69.0698, 81.0699, 83.0492, 83.0853, ...","[0.010357142857142856, 0.04642857142857143, 0....","[60.02260000000001, 118.02660000000003, 160.03...","[0.060714285714285714, 0.39285714285714285, 0....",CC1CC2(C(C1O)C=C(C(CC3C(C3(C)C)C=C(C2=O)C)OC(=...,0.0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,6,1102.5777,"[81.0334, 83.0491, 85.0224, 85.0284, 85.0333, ...","[0.14210526315789473, 0.2236842105263158, 0.01...",[],[],CC1C(C(C(C(O1)OC2C(OC(C(C2O)O)OCC3C(C(C(C(O3)O...,4.0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,10,472.2082,"[72.2736, 145.0759, 148.0868, 149.0707, 172.06...","[0.010697674418604652, 0.03488372093023256, 0....","[135.07919999999996, 148.0632, 165.08959999999...","[0.053488372093023255, 0.10465116279069768, 0....",CN1C2=C(C=C(C=C2)C(=O)N(CCC(=O)O)C3=CC=CC=N3)N...,0.0,1,...,0,0,0,0,0,1,0,0,0,0
4,4,11,657.3116,"[55.0542, 55.9818, 57.0334, 60.2456, 69.0335, ...","[0.018260869565217393, 0.013043478260869565, 0...","[158.094, 210.1282, 228.1352, 246.1485, 280.16...","[0.02217391304347826, 0.06086956521739131, 0.1...",CCCC(=O)OCC(C(C(CN1C2=C(C=C(C(=C2)C)C)N=C3C1=N...,1.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,237,490,268.1541,"[86.0599, 109.065, 109.1013, 121.101, 123.1168...","[0.10689655172413794, 0.04482758620689655, 0.0...","[18.01030000000003, 36.02120000000002, 46.0054...","[0.21724137931034482, 0.06206896551724138, 1.0...",CC1CCC2C1C(=O)OC(C2C)NC(=O)C3C(O3)C,0.0,0,...,0,0,0,0,0,0,0,0,0,0
238,238,491,411.3254,"[55.0544, 57.0701, 67.0544, 69.0699, 81.0699, ...","[0.0215625, 0.053125, 0.015, 0.34375, 0.046875...","[18.01060000000001, 88.08850000000001, 142.135...","[0.153125, 0.02125, 0.01875, 0.02625, 0.028437...",CC(C)C(C)C=CC(C)C1CCC2C1(CCC3=C2C(=O)C=C4C3(CC...,0.0,0,...,0,0,0,0,0,0,0,0,0,0
239,239,492,430.2432,"[50.4192, 52.9216, 74.1355, 81.0699, 86.4846, ...","[0.05555555555555555, 0.058333333333333334, 0....","[98.63150000000002, 179.0794, 197.0896, 214.55...","[0.06666666666666667, 0.2777777777777778, 1.0,...",CC1CCC(C2(C1=CC(CC2)C(=C)C(=O)O)C)OC3C(C(C(C(O...,1.0,1,...,0,0,0,0,0,0,0,0,0,0
240,240,495,578.2076,"[54.5814, 57.0338, 61.0286, 69.0335, 81.0331, ...","[0.01, 0.010769230769230769, 0.038461538461538...",[160.76359999999994],[0.011923076923076923],COC1=C2C(=CC(=C1OC3C(C(C(C(O3)CO)O)O)OC4C(C(C(...,2.0,1,...,0,0,0,0,0,0,0,0,0,0


adducts_v=df_valid['adduct'].to_list()

len(set(adducts_v+adducts))

list(set(adducts_v+adducts))

sorted_adducts=['[M+H-C2H4O2]+',#-60
  '[M-3H2O+H]+',#-54
  '[M-2H2O+H]+',#-36
  '[M-H2O+H]+',#-18
 '[M-NH3+H]+',#-17
  '[M]+',
  '[M+H]+',
  '[M+H+2i]+',
  '[M+NH3]+',#+17
  '[M+NH4]+',#+18
  '[M+Na]+',#+23
  '[M+H+CH3OH]',#+33
  '[M+K]+',#+39
  '[2M+H]+',
 '[2M+H+2i]+',
'[2M+NH4]+',
'[M-H+2Na]+',
 '[2M+Na]+',
'[2M+K]+',]






sorted_adducts.index('[M-NH3+H]+')

In [4]:
def get_adduct_num(adduct):
    sorted_adducts=['[M+H-C2H4O2]+',#-60
  '[M-3H2O+H]+',#-54
  '[M-2H2O+H]+',#-36
  '[M-H2O+H]+',#-18
 '[M-NH3+H]+',#-17
  '[M]+',
  '[M+H]+',
  '[M+H+2i]+',
  '[M+NH3]+',#+17
  '[M+NH4]+',#+18
  '[M+Na]+',#+23
  '[M+H+CH3OH]',#+33
  '[M+K]+',#+39
  '[2M+H]+',
 '[2M+H+2i]+',
'[2M+NH4]+',
'[M-H+2Na]+',
 '[2M+Na]+',
'[2M+K]+',]
    return sorted_adducts.index(adduct)


In [5]:
get_adduct_num('[M-NH3+H]+')

4

df_valid['adduct_enc'] = df_valid.apply(lambda x: get_adduct_num(x.adduct), axis=1)
df_valid

In [6]:
df_valid['mf'] = df_valid.apply(lambda x: get_mf(x.smiles_preprocessed), axis=1)

formulas=df_valid['mf'].to_list()+df_train['mf'].to_list()

In [7]:
from molmass import Formula

def number_of_atoms(form): 
    f= Formula(form.replace("-","").replace("+",""))
    MF=(f._elements)
    try:
        C=MF["C"][0]
    except:
        C=0
    try:
        H=MF["H"][0]
    except:
        H=0
    try:
        O=MF["O"][0]
    except:
        O=0
    try:
        N=MF["N"][0]
    except:
        N=0
    try:
        S=MF["S"][0]
    except:
        S=0
    try:
        I=MF["I"][0]
    except:
        I=0
    try:
        Br=MF["Br"][0]
    except:
        Br=0
    try:
        Cl=MF["Cl"][0]
    except:
        Cl=0
    try:
        F=MF["F"][0]
    except:
        F=0
    try:
        P=MF["P"][0]
    except:
        P=0
        
    return [C,H, O, N, S, I, Br,Cl, F,P]
    
 

In [8]:
df_valid['elements'] = df_valid.apply(lambda x:number_of_atoms(x.mf), axis=1)
df_valid

,Unnamed: 0,spectrum_id,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,smiles_preprocessed,num_of_sugars,Number of aliphatic carboxylic acids,...,Number of phosphoric ester groups,Number of piperdine rings,Number of primary amides,Number of pyridine rings,Number of quaternary nitrogens,Number of thioether,Number of thiazole rings,Number of unbranched alkanes of at least 4 members (excludes halogenated alkanes),mf,elements
0,0,3,719.2538,"[53.0387, 55.0179, 55.0545, 57.0338, 59.0492, ...","[0.017272727272727273, 0.01818181818181818, 0....",[314.15709999999996],[0.01818181818181818],CC1C(C(C(C(O1)OC2=C(OC3=C(C(=CC(=C3C2=O)O)O)CC...,2.0,0,...,0,0,0,0,0,0,0,0,C35H42O16,"[35, 42, 16, 0, 0, 0, 0, 0, 0, 0]"
1,1,5,499.2298,"[67.0543, 69.0698, 81.0699, 83.0492, 83.0853, ...","[0.010357142857142856, 0.04642857142857143, 0....","[60.02260000000001, 118.02660000000003, 160.03...","[0.060714285714285714, 0.39285714285714285, 0....",CC1CC2(C(C1O)C=C(C(CC3C(C3(C)C)C=C(C2=O)C)OC(=...,0.0,1,...,0,0,0,0,0,0,0,0,C26H36O8,"[26, 36, 8, 0, 0, 0, 0, 0, 0, 0]"
2,2,6,1102.5777,"[81.0334, 83.0491, 85.0224, 85.0284, 85.0333, ...","[0.14210526315789473, 0.2236842105263158, 0.01...",[],[],CC1C(C(C(C(O1)OC2C(OC(C(C2O)O)OCC3C(C(C(C(O3)O...,4.0,0,...,0,0,0,0,0,0,0,0,C54H84O22,"[54, 84, 22, 0, 0, 0, 0, 0, 0, 0]"
3,3,10,472.2082,"[72.2736, 145.0759, 148.0868, 149.0707, 172.06...","[0.010697674418604652, 0.03488372093023256, 0....","[135.07919999999996, 148.0632, 165.08959999999...","[0.053488372093023255, 0.10465116279069768, 0....",CN1C2=C(C=C(C=C2)C(=O)N(CCC(=O)O)C3=CC=CC=N3)N...,0.0,1,...,0,0,0,1,0,0,0,0,C25H25N7O3,"[25, 25, 3, 7, 0, 0, 0, 0, 0, 0]"
4,4,11,657.3116,"[55.0542, 55.9818, 57.0334, 60.2456, 69.0335, ...","[0.018260869565217393, 0.013043478260869565, 0...","[158.094, 210.1282, 228.1352, 246.1485, 280.16...","[0.02217391304347826, 0.06086956521739131, 0.1...",CCCC(=O)OCC(C(C(CN1C2=C(C=C(C(=C2)C)C)N=C3C1=N...,1.0,0,...,0,0,0,0,0,0,0,0,C33H44N4O10,"[33, 44, 10, 4, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,237,490,268.1541,"[86.0599, 109.065, 109.1013, 121.101, 123.1168...","[0.10689655172413794, 0.04482758620689655, 0.0...","[18.01030000000003, 36.02120000000002, 46.0054...","[0.21724137931034482, 0.06206896551724138, 1.0...",CC1CCC2C1C(=O)OC(C2C)NC(=O)C3C(O3)C,0.0,0,...,0,0,0,0,0,0,0,0,C14H21NO4,"[14, 21, 4, 1, 0, 0, 0, 0, 0, 0]"
238,238,491,411.3254,"[55.0544, 57.0701, 67.0544, 69.0699, 81.0699, ...","[0.0215625, 0.053125, 0.015, 0.34375, 0.046875...","[18.01060000000001, 88.08850000000001, 142.135...","[0.153125, 0.02125, 0.01875, 0.02625, 0.028437...",CC(C)C(C)C=CC(C)C1CCC2C1(CCC3=C2C(=O)C=C4C3(CC...,0.0,0,...,0,0,0,0,0,0,0,0,C28H42O2,"[28, 42, 2, 0, 0, 0, 0, 0, 0, 0]"
239,239,492,430.2432,"[50.4192, 52.9216, 74.1355, 81.0699, 86.4846, ...","[0.05555555555555555, 0.058333333333333334, 0....","[98.63150000000002, 179.0794, 197.0896, 214.55...","[0.06666666666666667, 0.2777777777777778, 1.0,...",CC1CCC(C2(C1=CC(CC2)C(=C)C(=O)O)C)OC3C(C(C(C(O...,1.0,1,...,0,0,0,0,0,0,0,0,C21H32O8,"[21, 32, 8, 0, 0, 0, 0, 0, 0, 0]"
240,240,495,578.2076,"[54.5814, 57.0338, 61.0286, 69.0335, 81.0331, ...","[0.01, 0.010769230769230769, 0.038461538461538...",[160.76359999999994],[0.011923076923076923],COC1=C2C(=CC(=C1OC3C(C(C(C(O3)CO)O)O)OC4C(C(C(...,2.0,1,...,0,0,0,0,0,0,0,0,C24H32O15,"[24, 32, 15, 0, 0, 0, 0, 0, 0, 0]"


In [9]:
ele=["C","H", "O"," N", "S", "I", "Br","Cl", "F","P"]
df_elements= pd.DataFrame(df_valid['elements'].to_list(), columns=ele)
df_elements

,C,H,O,N,S,I,Br,Cl,F,P
0,35,42,16,0,0,0,0,0,0,0
1,26,36,8,0,0,0,0,0,0,0
2,54,84,22,0,0,0,0,0,0,0
3,25,25,3,7,0,0,0,0,0,0
4,33,44,10,4,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
237,14,21,4,1,0,0,0,0,0,0
238,28,42,2,0,0,0,0,0,0,0
239,21,32,8,0,0,0,0,0,0,0
240,24,32,15,0,0,0,0,0,0,0


In [10]:
df_elements.to_numpy().max()

104

In [11]:
df_valid=df_valid.join(df_elements,how="outer")
df_valid

,Unnamed: 0,spectrum_id,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,smiles_preprocessed,num_of_sugars,Number of aliphatic carboxylic acids,...,C,H,O,N,S,I,Br,Cl,F,P
0,0,3,719.2538,"[53.0387, 55.0179, 55.0545, 57.0338, 59.0492, ...","[0.017272727272727273, 0.01818181818181818, 0....",[314.15709999999996],[0.01818181818181818],CC1C(C(C(C(O1)OC2=C(OC3=C(C(=CC(=C3C2=O)O)O)CC...,2.0,0,...,35,42,16,0,0,0,0,0,0,0
1,1,5,499.2298,"[67.0543, 69.0698, 81.0699, 83.0492, 83.0853, ...","[0.010357142857142856, 0.04642857142857143, 0....","[60.02260000000001, 118.02660000000003, 160.03...","[0.060714285714285714, 0.39285714285714285, 0....",CC1CC2(C(C1O)C=C(C(CC3C(C3(C)C)C=C(C2=O)C)OC(=...,0.0,1,...,26,36,8,0,0,0,0,0,0,0
2,2,6,1102.5777,"[81.0334, 83.0491, 85.0224, 85.0284, 85.0333, ...","[0.14210526315789473, 0.2236842105263158, 0.01...",[],[],CC1C(C(C(C(O1)OC2C(OC(C(C2O)O)OCC3C(C(C(C(O3)O...,4.0,0,...,54,84,22,0,0,0,0,0,0,0
3,3,10,472.2082,"[72.2736, 145.0759, 148.0868, 149.0707, 172.06...","[0.010697674418604652, 0.03488372093023256, 0....","[135.07919999999996, 148.0632, 165.08959999999...","[0.053488372093023255, 0.10465116279069768, 0....",CN1C2=C(C=C(C=C2)C(=O)N(CCC(=O)O)C3=CC=CC=N3)N...,0.0,1,...,25,25,3,7,0,0,0,0,0,0
4,4,11,657.3116,"[55.0542, 55.9818, 57.0334, 60.2456, 69.0335, ...","[0.018260869565217393, 0.013043478260869565, 0...","[158.094, 210.1282, 228.1352, 246.1485, 280.16...","[0.02217391304347826, 0.06086956521739131, 0.1...",CCCC(=O)OCC(C(C(CN1C2=C(C=C(C(=C2)C)C)N=C3C1=N...,1.0,0,...,33,44,10,4,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,237,490,268.1541,"[86.0599, 109.065, 109.1013, 121.101, 123.1168...","[0.10689655172413794, 0.04482758620689655, 0.0...","[18.01030000000003, 36.02120000000002, 46.0054...","[0.21724137931034482, 0.06206896551724138, 1.0...",CC1CCC2C1C(=O)OC(C2C)NC(=O)C3C(O3)C,0.0,0,...,14,21,4,1,0,0,0,0,0,0
238,238,491,411.3254,"[55.0544, 57.0701, 67.0544, 69.0699, 81.0699, ...","[0.0215625, 0.053125, 0.015, 0.34375, 0.046875...","[18.01060000000001, 88.08850000000001, 142.135...","[0.153125, 0.02125, 0.01875, 0.02625, 0.028437...",CC(C)C(C)C=CC(C)C1CCC2C1(CCC3=C2C(=O)C=C4C3(CC...,0.0,0,...,28,42,2,0,0,0,0,0,0,0
239,239,492,430.2432,"[50.4192, 52.9216, 74.1355, 81.0699, 86.4846, ...","[0.05555555555555555, 0.058333333333333334, 0....","[98.63150000000002, 179.0794, 197.0896, 214.55...","[0.06666666666666667, 0.2777777777777778, 1.0,...",CC1CCC(C2(C1=CC(CC2)C(=C)C(=O)O)C)OC3C(C(C(C(O...,1.0,1,...,21,32,8,0,0,0,0,0,0,0
240,240,495,578.2076,"[54.5814, 57.0338, 61.0286, 69.0335, 81.0331, ...","[0.01, 0.010769230769230769, 0.038461538461538...",[160.76359999999994],[0.011923076923076923],COC1=C2C(=CC(=C1OC3C(C(C(C(O3)CO)O)O)OC4C(C(C(...,2.0,1,...,24,32,15,0,0,0,0,0,0,0


In [12]:
df_valid=df_valid.drop(["elements","mf"], axis=1)
df_valid

,Unnamed: 0,spectrum_id,precursor_mz,mzs,intensities,loss_mzs,loss_intensities,smiles_preprocessed,num_of_sugars,Number of aliphatic carboxylic acids,...,C,H,O,N,S,I,Br,Cl,F,P
0,0,3,719.2538,"[53.0387, 55.0179, 55.0545, 57.0338, 59.0492, ...","[0.017272727272727273, 0.01818181818181818, 0....",[314.15709999999996],[0.01818181818181818],CC1C(C(C(C(O1)OC2=C(OC3=C(C(=CC(=C3C2=O)O)O)CC...,2.0,0,...,35,42,16,0,0,0,0,0,0,0
1,1,5,499.2298,"[67.0543, 69.0698, 81.0699, 83.0492, 83.0853, ...","[0.010357142857142856, 0.04642857142857143, 0....","[60.02260000000001, 118.02660000000003, 160.03...","[0.060714285714285714, 0.39285714285714285, 0....",CC1CC2(C(C1O)C=C(C(CC3C(C3(C)C)C=C(C2=O)C)OC(=...,0.0,1,...,26,36,8,0,0,0,0,0,0,0
2,2,6,1102.5777,"[81.0334, 83.0491, 85.0224, 85.0284, 85.0333, ...","[0.14210526315789473, 0.2236842105263158, 0.01...",[],[],CC1C(C(C(C(O1)OC2C(OC(C(C2O)O)OCC3C(C(C(C(O3)O...,4.0,0,...,54,84,22,0,0,0,0,0,0,0
3,3,10,472.2082,"[72.2736, 145.0759, 148.0868, 149.0707, 172.06...","[0.010697674418604652, 0.03488372093023256, 0....","[135.07919999999996, 148.0632, 165.08959999999...","[0.053488372093023255, 0.10465116279069768, 0....",CN1C2=C(C=C(C=C2)C(=O)N(CCC(=O)O)C3=CC=CC=N3)N...,0.0,1,...,25,25,3,7,0,0,0,0,0,0
4,4,11,657.3116,"[55.0542, 55.9818, 57.0334, 60.2456, 69.0335, ...","[0.018260869565217393, 0.013043478260869565, 0...","[158.094, 210.1282, 228.1352, 246.1485, 280.16...","[0.02217391304347826, 0.06086956521739131, 0.1...",CCCC(=O)OCC(C(C(CN1C2=C(C=C(C(=C2)C)C)N=C3C1=N...,1.0,0,...,33,44,10,4,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,237,490,268.1541,"[86.0599, 109.065, 109.1013, 121.101, 123.1168...","[0.10689655172413794, 0.04482758620689655, 0.0...","[18.01030000000003, 36.02120000000002, 46.0054...","[0.21724137931034482, 0.06206896551724138, 1.0...",CC1CCC2C1C(=O)OC(C2C)NC(=O)C3C(O3)C,0.0,0,...,14,21,4,1,0,0,0,0,0,0
238,238,491,411.3254,"[55.0544, 57.0701, 67.0544, 69.0699, 81.0699, ...","[0.0215625, 0.053125, 0.015, 0.34375, 0.046875...","[18.01060000000001, 88.08850000000001, 142.135...","[0.153125, 0.02125, 0.01875, 0.02625, 0.028437...",CC(C)C(C)C=CC(C)C1CCC2C1(CCC3=C2C(=O)C=C4C3(CC...,0.0,0,...,28,42,2,0,0,0,0,0,0,0
239,239,492,430.2432,"[50.4192, 52.9216, 74.1355, 81.0699, 86.4846, ...","[0.05555555555555555, 0.058333333333333334, 0....","[98.63150000000002, 179.0794, 197.0896, 214.55...","[0.06666666666666667, 0.2777777777777778, 1.0,...",CC1CCC(C2(C1=CC(CC2)C(=C)C(=O)O)C)OC3C(C(C(C(O...,1.0,1,...,21,32,8,0,0,0,0,0,0,0
240,240,495,578.2076,"[54.5814, 57.0338, 61.0286, 69.0335, 81.0331, ...","[0.01, 0.010769230769230769, 0.038461538461538...",[160.76359999999994],[0.011923076923076923],COC1=C2C(=CC(=C1OC3C(C(C(C(O3)CO)O)O)OC4C(C(C(...,2.0,1,...,24,32,15,0,0,0,0,0,0,0


In [13]:
group_array=df_valid.iloc[:, 8:]
group_array

,num_of_sugars,Number of aliphatic carboxylic acids,Number of aliphatic hydroxyl groups,Number of aliphatic hydroxyl groups excluding tert-OH,Number of N functional groups attached to aromatics,Number of Aromatic carboxylic acides,Number of aromatic nitrogens,Number of aromatic amines,Number of aromatic hydroxyl groups,Number of carboxylic acids,...,C,H,O,N,S,I,Br,Cl,F,P
0,2.0,0,5,5,0,0,0,0,2,0,...,35,42,16,0,0,0,0,0,0,0
1,0.0,1,1,1,0,0,0,0,0,1,...,26,36,8,0,0,0,0,0,0,0
2,4.0,0,11,11,0,0,0,0,0,0,...,54,84,22,0,0,0,0,0,0,0
3,0.0,1,0,0,1,0,3,0,0,1,...,25,25,3,7,0,0,0,0,0,0
4,1.0,0,0,0,0,0,4,1,0,0,...,33,44,10,4,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,0.0,0,0,0,0,0,0,0,0,0,...,14,21,4,1,0,0,0,0,0,0
238,0.0,0,1,1,0,0,0,0,0,0,...,28,42,2,0,0,0,0,0,0,0
239,1.0,1,4,4,0,0,0,0,0,1,...,21,32,8,0,0,0,0,0,0,0
240,2.0,1,7,7,0,0,0,0,0,1,...,24,32,15,0,0,0,0,0,0,0


In [14]:
group_array.to_numpy().max()

104.0

In [15]:
group_array.to_numpy().shape

(242, 70)

def make_float_targets(k):
  targets = np.zeros(k, dtype=np.float32)
  start = 1.0 / (2 * k)  # like 0.125
  delta = 1.0 / k        # like 0.250
  for i in range(k):
    targets[i] = start + (i * delta) 
  return targets

train_int=group_array.to_numpy()
ordinal=make_float_targets(65)
all_arr=[]
for sample in train_int:
    cache=[]
    for i in sample:
        cache.append(ordinal[int(i)])
    x=np.array(cache)
    all_arr.append(x)
    
train_y=np.stack(all_arr)   
train_y.shape  

In [16]:
#np.save('y1_all_HRMS_valid_24012023_cddd_mf.npy',train_y)

In [17]:
df_valid.to_csv('casmi_func_groups_2201_mf.tsv',sep='\t')